In [1]:
import random
import pandas as pd

# Kano Analysis
## Kano analysis is a framework for prioritizing product features from a backlog
## To conduct a Kano analysis, we need:
- A set of users to survey
- A small set of features to study
- Two questions about each feature:
    - Functional question: How do you feel when Feature X is available?
    - Dysfunctional question: How do you feel when Feature X is unavailable?
- A limited list of valid answers to each question

In [2]:
# simulate 100 users to be surveyed
users = [f'User {i}' for i in range(1, 101)]

# simulate 5 features to be studied
features = [f'Feature {i}' for i in range(1, 6)]

# specify the valid answers for each question
potentialAnswers = ['Like it', 'Expect it', 'Don\'t care', 'Can live with it', 'Dislike it']

In [3]:
# define the expected distribution of functional answers
functionalWeights = [0.35, 0.30, 0.10, 0.15, 0.10]

# define a weight matrix that approximates the relationship between functional and dysfunctional answers about the same feature
weights = pd.DataFrame(columns = potentialAnswers, index = potentialAnswers)
weights.loc['Like it'] = [0.05, 0.05, 0.2, 0.55, 0.15]
weights.loc['Expect it'] = [0.01, 0.01, 0.01, 0.45, 0.52]
weights.loc['Don\'t care'] = [0.01, 0.05, 0.6, 0.1, 0.24]
weights.loc['Can live with it'] = [0.49, 0.29, 0.01, 0.01, 0.2]
weights.loc['Dislike it'] = [0.85, 0.01, 0.01, 0.12, 0.01]
weights

,Like it,Expect it,Don't care,Can live with it,Dislike it
Like it,0.05,0.05,0.2,0.55,0.15
Expect it,0.01,0.01,0.01,0.45,0.52
Don't care,0.01,0.05,0.6,0.1,0.24
Can live with it,0.49,0.29,0.01,0.01,0.2
Dislike it,0.85,0.01,0.01,0.12,0.01


In [4]:
# simulate a set of answers for each user
answers = pd.DataFrame()
for feature in features:
    # perturb answer weights to simulate different appeal of different features
    functionalWeights_f = [wt * random.uniform(0.5, 1.5) for wt in functionalWeights]
    functionalWeights_f = [wt / sum(functionalWeights_f) for wt in functionalWeights_f]

    for user in users:        
        # simulate a variable response to each functional question
        functionalAnswer = random.choices(potentialAnswers, weights = functionalWeights_f, k = 1)[0]
        
        # expect an inverse response to the dysfunctional question - with some variation
        dysfunctionalAnswer = random.choices(potentialAnswers, weights = weights.loc[functionalAnswer], k = 1)[0]
        
        # combine these elements
        answer = pd.DataFrame([[user, feature, functionalAnswer, dysfunctionalAnswer]]
                              , columns = ['User', 'Feature', 'Functional Answer', 'Dysfunctional Answer'])
        
        # stack responses
        answers = pd.concat([answers, answer], ignore_index = True)
        
# convert answers to ordered categorical features for ease of evaluation
answers['Functional Answer'] = pd.Categorical(answers['Functional Answer']
                                              , categories = potentialAnswers, ordered = True)
answers['Dysfunctional Answer'] = pd.Categorical(answers['Dysfunctional Answer']
                                                 , categories = potentialAnswers, ordered = True)

In [5]:
# verify that answers are sensible
pd.pivot_table(answers, index = 'Functional Answer', columns = 'Dysfunctional Answer', aggfunc = 'count', values = 'User')

Dysfunctional Answer,Like it,Expect it,Don't care,Can live with it,Dislike it
Functional Answer,,,,,
Like it,12,12,29,90,27
Expect it,2,2,1,84,106
Don't care,1,1,33,3,13
Can live with it,28,18,2,0,5
Dislike it,25,0,0,5,1


In [6]:
answers.to_csv('./data/Survey Data.csv', index = False)